In [1]:
%config Completer.use_jedi = False

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from pathlib import Path
import time
from pysmFISH.pipeline import Pipeline
from pysmFISH import stitching
from pysmFISH import io

# List of all sections  

- LBEXP20200925_EEL_Mouse_-140um
- LBEXP20201104_EEL_Mouse_600um_2 
- LBEXP20201002_EEL_Mouse_1200um
- LBEXP20201107_EEL_Mouse_1810um 
- LBEXP20201014_EEL_Mouse_2420um 
- LBEXP20201010_EEL_Mouse_3000um
- LBEXP20201012_EEL_Mouse_3600um

__NB__: Analysis need to be run with the 'before room organization' microscope setup for the tiles_org

____________________

# LBEXP20200925_EEL_Mouse_-140um
2021-06-26

In [ ]:
experiment_fpath = Path('/fish/rawdata/LBEXP20200925_EEL_Mouse_-140um')
date_tag = time.strftime("%y%m%d_%H_%M_%S")
pipeline_run_name = date_tag + '_' + experiment_fpath.stem
run_type = 're-run'
parsing_type = 'no_parsing'
processing_engine = 'htcondor'

In [ ]:
%%time
running_pipeline = Pipeline(
        pipeline_run_name= pipeline_run_name,
        experiment_fpath= experiment_fpath,
        run_type= run_type,
        parsing_type= parsing_type,
        processing_engine= processing_engine,
        chunk_size=50,
        same_dot_radius_duplicate_dots = 100,
        save_bits_int=False,
        dataset_path='/fish/rawdata/LBEXP20200925_EEL_Mouse_-140um/210618_06_38_21_LBEXP20200925_EEL_Mouse_-140um_img_data_dataset.parquet'
        )

In [ ]:
%%time
running_pipeline.run_full()

__IMPORTANT__

The stitching didn't work nicely so I will remove the dots from the microscope stitched dots

In [ ]:
running_pipeline.determine_tiles_organization()

In [ ]:
%%time
running_pipeline.stitching_selected = 'microscope_stitched'

stitching.remove_duplicated_dots_graph(running_pipeline.experiment_fpath,running_pipeline.data.dataset,running_pipeline.tiles_org,
                        running_pipeline.hamming_distance,running_pipeline.same_dot_radius_duplicate_dots, 
                            running_pipeline.stitching_selected, running_pipeline.client)

In [ ]:
%%time
selected_Hdistance = 3 / running_pipeline.metadata['barcode_length']
stitching_selected = 'microscope_stitched'
io.simple_output_plotting(running_pipeline.experiment_fpath, stitching_selected, 
                        selected_Hdistance, running_pipeline.client,file_tag='cleaned')

In [ ]:
running_pipeline.cluster.close()
running_pipeline.client.close()

# LBEXP20201104_EEL_Mouse_600um_2

Using both-beads I got lower count and more tiles not nicely registered.
I will check if the number of small beads fluctuates more than with large, or maybe
the reconstruction of the image use a too large sigma and the dots overlay and
become messy

In [4]:
experiment_fpath = Path('/fish/rawdata/LBEXP20201104_EEL_Mouse_600um_2')
date_tag = time.strftime("%y%m%d_%H_%M_%S")
pipeline_run_name = date_tag + '_' + experiment_fpath.stem
run_type = 're-run'
parsing_type = 'no_parsing'
processing_engine = 'htcondor'

In [11]:
%%time
running_pipeline = Pipeline(
                        pipeline_run_name= pipeline_run_name,
                        experiment_fpath= experiment_fpath,
                        run_type= run_type,
                        parsing_type= parsing_type,
                        processing_engine= processing_engine,
                        chunk_size=50,
                        same_dot_radius_duplicate_dots = 100,
                        save_bits_int=False,
                        dataset_path='/fish/rawdata/LBEXP20201104_EEL_Mouse_600um_2/210627_20_37_35_LBEXP20201104_EEL_Mouse_600um_2_mod_img_data_dataset.parquet'
                )

distributed.deploy.adaptive_core - ERROR - Adaptive stopping due to error Timed out trying to connect to tcp://193.10.16.58:9610 after 10 s


CPU times: user 6.89 ms, sys: 6.54 ms, total: 13.4 ms
Wall time: 13.2 ms


In [6]:
%%time
running_pipeline.run_parsing_only()

distributed.worker - INFO - extra_processing_data already exist
distributed.worker - INFO - pipeline_config already exist
distributed.worker - INFO - output_figures already exist
distributed.worker - INFO - probes already exist
distributed.worker - INFO - logs already exist
distributed.worker - INFO - results already exist
distributed.worker - INFO - microscope_tiles_coords already exist


CPU times: user 752 ms, sys: 924 ms, total: 1.68 s
Wall time: 6 s


In [7]:
%%time 
running_pipeline.run_required_steps()

CPU times: user 12.4 s, sys: 45.6 s, total: 58 s
Wall time: 2min 17s


In [9]:
running_pipeline.data.dataset.loc[running_pipeline.data.dataset.channel == 'Europium', 'processing_type'] = 'both-beads'
running_pipeline.data.dataset.to_parquet('/fish/rawdata/LBEXP20201104_EEL_Mouse_600um_2/210627_20_37_35_LBEXP20201104_EEL_Mouse_600um_2_mod_img_data_dataset.parquet')

In [14]:
running_pipeline.cluster.close()
running_pipeline.client.close()

tornado.application - ERROR - Exception in callback functools.partial(<function TCPServer._handle_connection.<locals>.<lambda> at 0x7f833dc7ef70>, <Task finished name='Task-7050209' coro=<BaseTCPListener._handle_stream() done, defined at /home/simone/mini/envs/test_d/lib/python3.8/site-packages/distributed/comm/tcp.py:459> exception=ValueError('invalid operation on non-started TCPListener')>)
Traceback (most recent call last):
  File "/home/simone/mini/envs/test_d/lib/python3.8/site-packages/tornado/ioloop.py", line 741, in _run_callback
    ret = callback()
  File "/home/simone/mini/envs/test_d/lib/python3.8/site-packages/tornado/tcpserver.py", line 331, in <lambda>
    gen.convert_yielded(future), lambda f: f.result()
  File "/home/simone/mini/envs/test_d/lib/python3.8/site-packages/distributed/comm/tcp.py", line 465, in _handle_stream
    logger.debug("Incoming connection from %r to %r", address, self.contact_address)
  File "/home/simone/mini/envs/test_d/lib/python3.8/site-packages

In [ ]:
running_pipeline.run_full(resume=True)

In [13]:
running_pipeline.QC_registration_error_step()

UncompressError: Error while decompressing: invalid input

Modify registration type to large-beads

In [ ]:
%%time
running_pipeline.processing_barcoded_eel_step()

In [ ]:
running_pipeline.QC_registration_error_step()

In [ ]:
running_pipeline.running_functions

# LBEXP20210628_EEL_Mouse_448_1210um

In [4]:
experiment_fpath = Path('/fish/work_std/LBEXP20210628_EEL_Mouse_448_1210um')
date_tag = time.strftime("%y%m%d_%H_%M_%S")
pipeline_run_name = date_tag + '_' + experiment_fpath.stem
run_type = 'new'
parsing_type = 'original'
processing_engine = 'htcondor'

In [5]:
%%time
running_pipeline = Pipeline(
                        pipeline_run_name= pipeline_run_name,
                        experiment_fpath= experiment_fpath,
                        run_type= run_type,
                        parsing_type= parsing_type,
                        processing_engine= processing_engine,
                        chunk_size=50,
                        same_dot_radius_duplicate_dots = 100,
                        save_bits_int=False,
                        dataset_path = '/fish/work_std/LBEXP20210628_EEL_Mouse_448_1210um/210715_01_27_32_LBEXP20210628_EEL_Mouse_448_1210um_img_data_dataset.parquet',
                )

CPU times: user 99 µs, sys: 64 µs, total: 163 µs
Wall time: 168 µs


In [ ]:
%%time
running_pipeline.run_full()

In [13]:
running_pipeline.cluster.close()
running_pipeline.client.close()

distributed.deploy.adaptive_core - ERROR - Adaptive stopping due to error in <closed TCP>: Stream is closed: while trying to call remote method 'adaptive_target'


In [6]:
running_pipeline.prepare_processing_dataset_step()

In [10]:
running_pipeline.data.dataset.loc[running_pipeline.data.dataset.channel == 'Cy5','processing_type']

0        fish
1        fish
2        fish
3        fish
4        fish
         ... 
24733    fish
24734    fish
24735    fish
24736    fish
24737    fish
Name: processing_type, Length: 12768, dtype: object

In [ ]:
%%time
running_pipeline.run_parsing_only()

__IMPORTANT__

After parsing the dataset need to be fixed because there is an error in the pkl and yaml file
indicating

StitchingChannel as Europium_Cy5

I corrected the yaml to:
    
StitchingChannel as Europium_Cy5  
Pipeline: eel-mouse-brain  
robe_FASTA_name: mBrain.fasta

In [ ]:
running_pipeline.data.dataset.loc[running_pipeline.data.dataset.channel == 'Europium','processing_type']

In [ ]:
%%time 
running_pipeline.run_required_steps()

In [ ]:
%%time
running_pipeline.processing_barcoded_eel_step()

In [ ]:
%%time
running_pipeline.stitch_and_remove_dots_eel_graph_step()

In [ ]:
__IMPORTANT__

The stitching didn't work nicely so I will remove the dots from the microscope stitched dots

In [ ]:
%%time
running_pipeline.determine_tiles_organization()

In [ ]:
%%time
running_pipeline.stitching_selected = 'microscope_stitched'

stitching.remove_duplicated_dots_graph(running_pipeline.experiment_fpath,running_pipeline.data.dataset,running_pipeline.tiles_org,
                        running_pipeline.hamming_distance,running_pipeline.same_dot_radius_duplicate_dots, 
                            running_pipeline.stitching_selected, running_pipeline.client)

In [ ]:
%%time
selected_Hdistance = 3 / running_pipeline.metadata['barcode_length']
stitching_selected = 'microscope_stitched'
io.simple_output_plotting(running_pipeline.experiment_fpath, stitching_selected, 
                        selected_Hdistance, running_pipeline.client,file_tag='cleaned')

In [ ]:
%%time
running_pipeline.QC_registration_error_step()

In [ ]:
running_pipeline.running_functions

In [ ]:
# ----------------------------------------------------------------
# GENERATE OUTPUT FOR PLOTTING
selected_Hdistance = 3 / self.metadata['barcode_length']
stitching_selected = 'microscope_stitched'
io.simple_output_plotting(self.experiment_fpath, stitching_selected, 
                        selected_Hdistance, self.client,file_tag='decoded')
# ----------------------------------------------------------------  

In [ ]:
# LBEXP20201002_EEL_Mouse_1200um

In [ ]:
running_pipeline.same_dot_radius_duplicate_dots = 20

In [ ]:
%%time 
running_pipeline.stitch_and_remove_dots_eel_graph_step()

In [ ]:
# ----------------------------------------------------------------
# GENERATE OUTPUT FOR PLOTTING
selected_Hdistance = 3 / 16
stitching_selected = 'microscope_stitched'
io.simple_output_plotting(experiment_fpath, stitching_selected, 
                        selected_Hdistance, running_pipeline.client,file_tag='decoded')
# ----------------------------------------------------------------  

In [ ]:
from pysmFISH import stitching

In [ ]:
import pandas as pd

In [ ]:
from dask import dataframe as dd

In [ ]:
running_pipeline.determine_tiles_organization()

In [ ]:
def stmps(fname,tile_corners_coords_pxl):
    decoded_df = pd.read_parquet(fname)
    stitching.stitch_using_coords_general(decoded_df,tile_corners_coords_pxl,tag='microscope_stitched')

In [ ]:
fnames = list((running_pipeline.experiment_fpath / 'results').glob('*_decoded_fov_*'))
futures = []
for fname in fnames:
    print(fname)
    stitching.stitch_using_coords_general(decoded_df,running_pipeline.tile_corners_coords_pxl,tag='microscope_stitched')

In [ ]:
data = pd.read_parquet('/fish/rawdata/LBEXP20200622_EEL_Mouse_0um/results/LBEXP20200622_EEL_Mouse_0um_decoded_fov_10.parquet')

In [ ]:
data.columns

In [ ]:
running_pipeline.client.gather(futures)

In [ ]:
del futures

In [ ]:
decoded_df = pd.read_parquet(fname)

In [ ]:
decoded_df

In [ ]:
# ----------------------------------------------------------------
# GENERATE OUTPUT FOR PLOTTING
selected_Hdistance = 3 / 16
stitching_selected = 'microscope_stitched'
io.simple_output_plotting(experiment_fpath, stitching_selected, 
                        selected_Hdistance, running_pipeline.client,file_tag='decoded')
# ----------------------------------------------------------------  

In [ ]:
[k for k in running_pipeline.analysis_parameters.keys() if 'beads' in k]